In [28]:
import pandas as pd
import numpy as np

In [2]:
folder = '.\\input_files\\'

file = 'documents_standard-charges_20-0479568_Essentia-Health-Ada_standardcharges.csv'

In [10]:
df = pd.read_csv(folder + file, dtype=str, encoding='ansi', skiprows=4)

In [13]:
df.rename(columns={
    'EAP Code (Internal Use)': 'local_code',
    'CPT/HCPCS Code': 'code',
    'Procedure Description (Charge)': 'description',
    'Raw 11 Digit NDC or Primary Ext Id': 'ndc',
    'Revenue Code': 'rev_code'
}, inplace=True)

In [20]:
cols = df.columns.tolist()
id_vars = cols[:5]
value_vars = cols[5:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [24]:
mapping = {
    'CDM Price': 'gross',
    'Discounted Self Pay/Cash Price': 'cash',
    'Inpatient De-Identified Minimum Negoticated Rate': 'min',
    'Inpatient De-Identified Maximum Negoticated Rate': 'max',
    'Outpatient De-Identified Minimum Negoticated Rate': 'min',
    'Outpatient De-Identified Maximum Negoticated Rate': 'max'
}

df['payer_category'] = df['payer_name'].map(mapping).fillna('payer')

In [26]:
df.loc[df['payer_name'].str.lower().str.contains('inpatient'), 'setting'] = 'inpatient'

In [31]:
df['setting'] = np.where(
    df['payer_name'].str.lower().str.contains('inpatient'), 'inpatient',
    np.where(
        df['payer_name'].str.lower().str.contains('outpatient'), 'outpatient', pd.NA)
)

In [34]:
df['rev_code'] = df['rev_code'].str.zfill(4)

In [36]:
df.loc[df['rev_code'].isna()]

,local_code,code,description,ndc,rev_code,payer_name,standard_charge,payer_category,setting
3526,NaN,NaN,NaN,NaN,NaN,CDM Price,NaN,gross,NaN
3527,Per Diem Rate,NaN,NaN,NaN,NaN,CDM Price,Median Charges,gross,NaN
3528,NaN,NaN,NaN,NaN,NaN,CDM Price,NaN,gross,NaN
3529,NaN,NaN,NaN,NaN,NaN,CDM Price,NaN,gross,NaN
3530,DRG Type,DRG Code,DRG Description,NaN,NaN,CDM Price,Median Charges,gross,NaN
...,...,...,...,...,...,...,...,...,...
860705,EAPG,2004,INCIDENTAL MINOR DIAGNOSTIC TESTS,NaN,NaN,WPS-Outpatient,NaN,payer,outpatient
860706,EAPG,2005,INCIDENTAL MEDICAL VISIT SERVICES,NaN,NaN,WPS-Outpatient,NaN,payer,outpatient
860707,EAPG,2006,INCIDENTAL IMPLANTABLE OR OTHER RELATED DEVICES,NaN,NaN,WPS-Outpatient,NaN,payer,outpatient
860708,EAPG,2007,INCIDENTAL BEHAVIORAL HEALTH SERVICES,NaN,NaN,WPS-Outpatient,NaN,payer,outpatient
